# 현대자동차 뉴스 그룹 미디어에서 '제네시스' 블로그 검색 결과 크롤링

- https://news.hmgjournal.com/

In [27]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By   # 특정 요소 존재 여부 확인
from selenium.webdriver.support.ui import WebDriverWait # 웹 페이지 로딩 완료될 때까지 대기
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys # key event 발생 시킴

import time

# Exception
from selenium.common.exceptions import TimeoutException, NoSuchElementException

import re # 특수 문자 제거위해 참조

## 현대 저널 페이지에서 검색

In [28]:
path = './driver/chromedriver.exe'
url = 'https://news.hmgjournal.com/Search-Result/List?newsearchKey=%EC%A0%9C%EB%84%A4%EC%8B%9C%EC%8A%A4&searchKey=&searchMenuId=AL'

In [29]:
driver = webdriver.Chrome(path)     # 크롬 브라우저 제어할 수 있는 객체, 웹브라우저 실행
driver
driver.get(url)

## 블로그 탭으로 이동

In [30]:
blogTab = driver.find_element_by_css_selector('#contents > ul > li:nth-child(2) > a')
blogTab.click()

## 검색 결과 크롤링

### 1. 첫번째 요소만 크롤링(테스트)

In [8]:
# category
category = driver.find_element_by_css_selector('#listImg > li.firstContent > a > strong').text
category

'Tech'

In [10]:
# title
title = driver.find_element_by_css_selector('#listImg > li.firstContent > a > div > span').text
title

'역동적인 아름다움으로 완성된 럭셔리 중형 SUV, 제네시스 GV70의 외장 디자인 개발 스토리'

In [11]:
# link
link = driver.find_element_by_css_selector('#listImg > li.firstContent > a').get_attribute('href')
link

'https://news.hmgjournal.com/Tech/%EC%A0%9C%EB%84%A4%EC%8B%9C%EC%8A%A4-GV70-%EC%99%B8%EC%9E%A5%EB%94%94%EC%9E%90%EC%9D%B8-%EA%B0%9C%EB%B0%9C%EC%8A%A4%ED%86%A0%EB%A6%AC'

In [86]:
# 더보기 버튼 클릭
btnMore = driver.find_element_by_css_selector('#contents > article > div > div > a')
btnMore.click()

### 2. 더보기 버튼 안나올 때까지 클릭

In [82]:
def clickBtnMore(pCssSelector) :
    
    cntClick = 0
    delay = 10
    
    while True :
        
        cntClick += 1
        print(cntClick)
        
        try : 
            
            btnMore = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, pCssSelector)))
            btnMore.send_keys(Keys.ENTER) 
            time.sleep(1)
            

        except TimeoutException as toe:
            print('TimeoutException : ', toe)          
            return   

        except Exception as e :
            print('예외 발생 :', e)
            return

In [88]:
btnMoreCss = '#contents > article> div.tab-cons-area > div.btn-wrap > a.btn-more'
clickBtnMore(btnMoreCss) # 총 285번

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
예외 발생 : Message: element not interactable
  (Session info: chrome=87.0.4280.88)



### 3. 전체 요소 크롤링

In [89]:
liList = driver.find_elements_by_css_selector('#contents > article > div > ul > li')
len(liList)

440

In [90]:
liList[0].find_element_by_css_selector( 'a').get_attribute('href')

'https://news.hmgjournal.com/Tech/%EC%A0%9C%EB%84%A4%EC%8B%9C%EC%8A%A4-GV70-%EC%99%B8%EC%9E%A5%EB%94%94%EC%9E%90%EC%9D%B8-%EA%B0%9C%EB%B0%9C%EC%8A%A4%ED%86%A0%EB%A6%AC'

In [97]:
def getElement() :
    liList = driver.find_elements_by_css_selector('#contents > article > div > ul > li')
    
    cnt = 0
    
    for li in liList :
        
        cnt+=1
        print(cnt)
        try : 
            categoryList.append(li.find_element_by_css_selector('a > strong').text) #listImg > li:nth-child(385) > a > strong
            titleList.append(li.find_element_by_css_selector('a > div > span').text)
            linkList.append(li.find_element_by_css_selector('a').get_attribute('href'))
        except Exception as e:
            print ('exception : ', e)

In [98]:
categoryList = []
titleList = []
linkList = []

getElement()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [99]:
print(len(categoryList), len(titleList), len(linkList))

439 439 439


In [107]:
genBlogDf = pd.DataFrame(data = {'magazine': 'HMGJ',
                                 'type' : 'B',
                                 'category' : categoryList,
                                 'title': titleList,
                                 'link' : linkList})

In [108]:
genBlogDf.head()

,magazine,type,category,title,link
0,HMGJ,B,Tech,"역동적인 아름다움으로 완성된 럭셔리 중형 SUV, 제네시스 GV70의 외장 디자인 ...",https://news.hmgjournal.com/Tech/%EC%A0%9C%EB%...
1,HMGJ,B,Group,"제네시스의 두 번째 SUV, GV70의 디자인에 대한 해외 반응",https://news.hmgjournal.com/Group-Story/%EC%A0...
2,HMGJ,B,Group,"제네시스의 가장 역동적인 스포츠 세단, 더 뉴 G70 출시",https://news.hmgjournal.com/Group-Story/%EC%A0...
3,HMGJ,B,Group,"제네시스 '더 뉴 G70' 디자인 공개, 해외 언론과 소비자들의 뜨거운 반응",https://news.hmgjournal.com/Group-Story/%EC%A0...
4,HMGJ,B,Group,"브랜드 최대 규모의 특별 체험 공간, 제네시스 수지",https://news.hmgjournal.com/Group-Story/%EB%B8...


In [110]:
genBlogDf.to_csv('./data/hmgj_genesis_crawling.csv', sep = '|', encoding = 'utf-8', index = False)

## 블로그 글 본문 크롤링

In [62]:
# date
date = driver.find_element_by_css_selector('#print > article > div.tit_view > span').text[:10]
date

'2020/12/04'

In [9]:
# content
content = driver.find_element_by_css_selector('#view-cont > div > div:nth-child(1)').text
content

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#view-cont > div > div:nth-child(1)"}
  (Session info: chrome=87.0.4280.88)


In [66]:
# img alt
imgAlt = driver.find_element_by_css_selector('#view-cont > div > img:nth-child(5)').get_attribute('alt')
imgAlt

'제네시스 GV70의 외장 디자인 개발 스토리'

In [35]:
def getElement() :
    
    count = 0
    
#     driver = webdriver.Chrome(path)   
    
    for link in genBlogDf['link'] :
        
        description = ''
        img = ''
        
        driver.get(link)
        
        count += 1
        print(count)
        
        dateList.append(driver.find_element_by_css_selector('#print > article > div.tit_view > span').text[:10])        
        divList = driver.find_elements_by_css_selector('#view-cont > div > div')
        imgList = driver.find_elements_by_css_selector('#view-cont > div > img')
        
        for div in divList :
            className = div.get_attribute("class")
            
            if className == 'discription' :
                description += ', ' + div.find_element_by_css_selector('span').text
                
        for imgTag in imgList :
            img += ', ' +imgTag.get_attribute('alt')
        
        desList.append(description)
        imgAltList.append(img)

In [36]:
dateList = []
desList = []
imgAltList = []

getElement()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [40]:
contentList = []
for i in range(len(desList)) :
    contentList.append(desList[i] + ', ' +imgAltList[i])

In [45]:
contentList

[', GV70는 제네시스 브랜드의 첫 번째 중형 SUV 모델이자 브랜드의 다섯 번째 모델로, 제네시스의 라인업을 확장하는 새로운 차종이다. GV70는 두 줄 디자인의 쿼드램프와 크레스트 그릴(라디에이터 그릴) 등 제네시스 고유의 디자인 DNA를 계승하는 한편, 후륜구동의 장점을 극대화한 세련된 비율까지 구현한 고급스럽고 역동적인 외관을 갖췄다. 그렇다면 이런 GV70의 디자인은 어떻게 만들어졌을까? 외관 디자인을 담당한 김기억 책임연구원으로부터 GV70 외장 디자인 개발과 관련된 이야기를 들어봤다., Q. 제네시스 브랜드의 디자인 전략을 GV70에 어떻게 반영했나?\n\n제네시스 브랜드의 디자인 기조는 ‘역동적인 우아함(Athletic Elegance)’을 바탕으로 한다. 그리고 모델마다 성격을 고려해 역동성과 우아함이 차지하는 비율을 다르게 연출하고 있다. G70와 GV70는 역동성을, 모델 라인업 중심에 위치한 G80와 GV80는 역동성과 우아함의 절묘한 균형을, 플래그십 세단 G90는 우아함을 강조한다. 이것이 제네시스 브랜드가 추구하는 디자인 전략의 핵심이다. GV70는 제네시스(Genesis)의 다재다능한(Versatile) 차량이라는 의미가 담긴 이니셜 ‘GV’와 중형 차급을 뜻하는 ‘70’를 결합한 차명에서 알 수 있듯이 럭셔리 중형 SUV 시장을 겨냥하는 모델이다., Q. 전면부 디자인의 특징은 무엇인가?\n\n전면부에서는 제네시스의 윙 엠블럼을 쿼드램프와 크레스트 그릴로 형상화한 윙 페이스(Wing Face) 디자인이 가장 큰 특징이다. 쿼드램프와 크레스트 그릴을 동일한 높이로 배열해 와이드한 느낌의 전면부 이미지를 구현했다. 또한 볼륨감과 입체감을 강조한 지-매트릭스(G-Matrix) 패턴을 라디에이터 그릴과 인테이크 그릴(범퍼 흡기구)에 적용해 제네시스만의 아이덴티티와 스포티한 감성을 전달한다. 아울러 앞범퍼 하단의 스키드 플레이트에서 시작되는 크롬 몰딩을 도어 하단까지 연장해 차량의 존재감을 키웠다. 이는 SUV의 성격과 스포티한 스타일

In [116]:
dateList = [date.replace('/', '-') for date in dateList]

In [117]:
genBlogDf['date'] = dateList

In [46]:
genBlogDf['contents'] = contentList

In [47]:
genBlogDf.head()

,magazine,type,category,title,link,date,contents
0,HMGJ,B,Tech,"역동적인 아름다움으로 완성된 럭셔리 중형 SUV, 제네시스 GV70의 외장 디자인 ...",https://news.hmgjournal.com/Tech/%EC%A0%9C%EB%...,2020-12-04,", GV70는 제네시스 브랜드의 첫 번째 중형 SUV 모델이자 브랜드의 다섯 번째 ..."
1,HMGJ,B,Group,"제네시스의 두 번째 SUV, GV70의 디자인에 대한 해외 반응",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-11-03,", , 제네시스의 두 번째 SUV, GV70의 디자인에 대한 해외 반응, 제네시스의..."
2,HMGJ,B,Group,"제네시스의 가장 역동적인 스포츠 세단, 더 뉴 G70 출시",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-10-20,","
3,HMGJ,B,Group,"제네시스 '더 뉴 G70' 디자인 공개, 해외 언론과 소비자들의 뜨거운 반응",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-09-23,", , 제네시스 더 뉴 G70 디자인 공개, 해외 언론과 소비자들의 뜨거운 반응, ..."
4,HMGJ,B,Group,"브랜드 최대 규모의 특별 체험 공간, 제네시스 수지",https://news.hmgjournal.com/Group-Story/%EB%B8...,2020-08-07,","


In [12]:
genBlogDf.to_csv('./data/hmgj_genesis_crawling.csv', sep = '|', encoding = 'utf-8', index = False)

In [5]:
genBlogDf = pd.read_csv('./data/hmgj_genesis_crawling.csv', sep = '|')

In [6]:
genBlogDf.head()

,magazine,type,category,title,link,date,contents
0,HMGJ,B,Tech,"역동적인 아름다움으로 완성된 럭셔리 중형 SUV, 제네시스 GV70의 외장 디자인 ...",https://news.hmgjournal.com/Tech/%EC%A0%9C%EB%...,2020-12-04,", GV70는 제네시스 브랜드의 첫 번째 중형 SUV 모델이자 브랜드의 다섯 번째 ..."
1,HMGJ,B,Group,"제네시스의 두 번째 SUV, GV70의 디자인에 대한 해외 반응",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-11-03,", , 제네시스의 두 번째 SUV, GV70의 디자인에 대한 해외 반응, 제네시스의..."
2,HMGJ,B,Group,"제네시스의 가장 역동적인 스포츠 세단, 더 뉴 G70 출시",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-10-20,","
3,HMGJ,B,Group,"제네시스 '더 뉴 G70' 디자인 공개, 해외 언론과 소비자들의 뜨거운 반응",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-09-23,", , 제네시스 더 뉴 G70 디자인 공개, 해외 언론과 소비자들의 뜨거운 반응, ..."
4,HMGJ,B,Group,"브랜드 최대 규모의 특별 체험 공간, 제네시스 수지",https://news.hmgjournal.com/Group-Story/%EB%B8...,2020-08-07,","


In [7]:
len(genBlogDf.loc[2, 'contents'].replace(' ', ''))

1

In [8]:
dropIdxList = []
for i in range(len(genBlogDf)) :
    if len(genBlogDf.loc[i, 'contents'].replace(' ', '')) == 1 :
        dropIdxList.append(i)    

In [9]:
dropIdxList

[2,
 4,
 5,
 7,
 8,
 9,
 12,
 14,
 16,
 18,
 20,
 24,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 57,
 58,
 60,
 61,
 62,
 63,
 65,
 67,
 68,
 69,
 70,
 72,
 74,
 75,
 76,
 82,
 85,
 88,
 89,
 90,
 93,
 99,
 102,
 103,
 104,
 105,
 106,
 107,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 122,
 123,
 124,
 125,
 126,
 127,
 133,
 134,
 137,
 140,
 141,
 144,
 148,
 149,
 150,
 151,
 154,
 156,
 157,
 158,
 159,
 161,
 163,
 165,
 166,
 167,
 168,
 169,
 171,
 172,
 173,
 174,
 175,
 179,
 181,
 183,
 184,
 186,
 188,
 189,
 190,
 191,
 192,
 193,
 195,
 196,
 198,
 199,
 200,
 203,
 204,
 206,
 209,
 210,
 211,
 212,
 213,
 215,
 216,
 217,
 218,
 219,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 233,
 234,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 252,
 253,
 255,
 256,
 258,
 259,
 260,
 262,
 264,
 265,
 266,
 267,
 270,
 271,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 27

In [10]:
genBlogDf = genBlogDf.drop(dropIdxList)
genBlogDf.head(10)

,magazine,type,category,title,link,date,contents
0,HMGJ,B,Tech,"역동적인 아름다움으로 완성된 럭셔리 중형 SUV, 제네시스 GV70의 외장 디자인 ...",https://news.hmgjournal.com/Tech/%EC%A0%9C%EB%...,2020-12-04,", GV70는 제네시스 브랜드의 첫 번째 중형 SUV 모델이자 브랜드의 다섯 번째 ..."
1,HMGJ,B,Group,"제네시스의 두 번째 SUV, GV70의 디자인에 대한 해외 반응",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-11-03,", , 제네시스의 두 번째 SUV, GV70의 디자인에 대한 해외 반응, 제네시스의..."
3,HMGJ,B,Group,"제네시스 '더 뉴 G70' 디자인 공개, 해외 언론과 소비자들의 뜨거운 반응",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-09-23,", , 제네시스 더 뉴 G70 디자인 공개, 해외 언론과 소비자들의 뜨거운 반응, ..."
6,HMGJ,B,Group,"오트 쿠튀르의 우아함을 담다, 제네시스 G90 스페셜 에디션 스타더스트",https://news.hmgjournal.com/Group-Story/%EC%98...,2020-07-03,", , 제네시스 G90 스타더스트, 제네시스 G90 스타더스트, 제네시스 G90 스..."
10,HMGJ,B,Tech,"제네시스 G80, 디자이너가 말하는 역동적인 우아함",https://news.hmgjournal.com/Tech/%EC%A0%9C%EB%...,2020-03-30,", , 제네시스_G80_디자이너가_말하는_역동적인_우아함, 제네시스_G80_디자이너..."
11,HMGJ,B,Group,"제네시스 신형 G80 디자인, 해외 호평 쏟아지는 이유는",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-03-10,", , 제네시스 The All-new G80, 전 세계 언론 매체와 소비자들의 주목..."
13,HMGJ,B,Group,"제네시스, 우수한 품질로 프리미엄 브랜드의 자질을 입증하다",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-02-28,", , 제네시스, 우수한 품질로 프리미엄 브랜드의 자질을 입증하다, 제네시스, 우수..."
15,HMGJ,B,Group,"나만을 위한 GV80, 제네시스의 맞춤형 주문 프로그램 '유어 제네시스'",https://news.hmgjournal.com/Group-Story/%EB%A7...,2020-02-12,", , 맞춤형_주문_프로그램_유어제네시스, 맞춤형_주문_프로그램_유어제네시스, 맞춤..."
17,HMGJ,B,Tech,컬러 디자이너가 말하는 제네시스 GV80의 매력,https://news.hmgjournal.com/Tech/%EC%BB%AC%EB%...,2020-01-16,", , 컬러 디자이너가 말하는 제네시스 GV80의 매력, 컬러 디자이너가 말하는 제..."
19,HMGJ,B,Tech,"우아한 차체에 빚어진 강인함, 디자이너가 말하는 제네시스 GV80",https://news.hmgjournal.com/Tech/%EC%9A%B0%EC%...,2020-01-15,", , GV80-EXT-Designer, GV80-EXT-Designer, GV80..."


In [11]:
genBlogDf = genBlogDf.reset_index(drop = True)
genBlogDf.head(10)

,magazine,type,category,title,link,date,contents
0,HMGJ,B,Tech,"역동적인 아름다움으로 완성된 럭셔리 중형 SUV, 제네시스 GV70의 외장 디자인 ...",https://news.hmgjournal.com/Tech/%EC%A0%9C%EB%...,2020-12-04,", GV70는 제네시스 브랜드의 첫 번째 중형 SUV 모델이자 브랜드의 다섯 번째 ..."
1,HMGJ,B,Group,"제네시스의 두 번째 SUV, GV70의 디자인에 대한 해외 반응",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-11-03,", , 제네시스의 두 번째 SUV, GV70의 디자인에 대한 해외 반응, 제네시스의..."
2,HMGJ,B,Group,"제네시스 '더 뉴 G70' 디자인 공개, 해외 언론과 소비자들의 뜨거운 반응",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-09-23,", , 제네시스 더 뉴 G70 디자인 공개, 해외 언론과 소비자들의 뜨거운 반응, ..."
3,HMGJ,B,Group,"오트 쿠튀르의 우아함을 담다, 제네시스 G90 스페셜 에디션 스타더스트",https://news.hmgjournal.com/Group-Story/%EC%98...,2020-07-03,", , 제네시스 G90 스타더스트, 제네시스 G90 스타더스트, 제네시스 G90 스..."
4,HMGJ,B,Tech,"제네시스 G80, 디자이너가 말하는 역동적인 우아함",https://news.hmgjournal.com/Tech/%EC%A0%9C%EB%...,2020-03-30,", , 제네시스_G80_디자이너가_말하는_역동적인_우아함, 제네시스_G80_디자이너..."
5,HMGJ,B,Group,"제네시스 신형 G80 디자인, 해외 호평 쏟아지는 이유는",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-03-10,", , 제네시스 The All-new G80, 전 세계 언론 매체와 소비자들의 주목..."
6,HMGJ,B,Group,"제네시스, 우수한 품질로 프리미엄 브랜드의 자질을 입증하다",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-02-28,", , 제네시스, 우수한 품질로 프리미엄 브랜드의 자질을 입증하다, 제네시스, 우수..."
7,HMGJ,B,Group,"나만을 위한 GV80, 제네시스의 맞춤형 주문 프로그램 '유어 제네시스'",https://news.hmgjournal.com/Group-Story/%EB%A7...,2020-02-12,", , 맞춤형_주문_프로그램_유어제네시스, 맞춤형_주문_프로그램_유어제네시스, 맞춤..."
8,HMGJ,B,Tech,컬러 디자이너가 말하는 제네시스 GV80의 매력,https://news.hmgjournal.com/Tech/%EC%BB%AC%EB%...,2020-01-16,", , 컬러 디자이너가 말하는 제네시스 GV80의 매력, 컬러 디자이너가 말하는 제..."
9,HMGJ,B,Tech,"우아한 차체에 빚어진 강인함, 디자이너가 말하는 제네시스 GV80",https://news.hmgjournal.com/Tech/%EC%9A%B0%EC%...,2020-01-15,", , GV80-EXT-Designer, GV80-EXT-Designer, GV80..."


In [18]:
len(genBlogDf)

332

In [31]:
contentList = []
for i in range(len(genBlogDf)):
    innerList = []    

    for content in genBlogDf.loc[i,'contents'].split("', '") :
        if ('제네시스' in content ) or ('Genesis' in content) or ('GENESIS' in content) or ('GV80' in content) or ('G80' in content) or ('GV70' in content) :
            innerList.append(content)
            
        contentList.append(innerList)

In [17]:
len(contentList)

332

In [33]:
contentList

[[', GV70는 제네시스 브랜드의 첫 번째 중형 SUV 모델이자 브랜드의 다섯 번째 모델로, 제네시스의 라인업을 확장하는 새로운 차종이다. GV70는 두 줄 디자인의 쿼드램프와 크레스트 그릴(라디에이터 그릴) 등 제네시스 고유의 디자인 DNA를 계승하는 한편, 후륜구동의 장점을 극대화한 세련된 비율까지 구현한 고급스럽고 역동적인 외관을 갖췄다. 그렇다면 이런 GV70의 디자인은 어떻게 만들어졌을까? 외관 디자인을 담당한 김기억 책임연구원으로부터 GV70 외장 디자인 개발과 관련된 이야기를 들어봤다., Q. 제네시스 브랜드의 디자인 전략을 GV70에 어떻게 반영했나? 제네시스 브랜드의 디자인 기조는 ‘역동적인 우아함(Athletic Elegance)’을 바탕으로 한다. 그리고 모델마다 성격을 고려해 역동성과 우아함이 차지하는 비율을 다르게 연출하고 있다. G70와 GV70는 역동성을, 모델 라인업 중심에 위치한 G80와 GV80는 역동성과 우아함의 절묘한 균형을, 플래그십 세단 G90는 우아함을 강조한다. 이것이 제네시스 브랜드가 추구하는 디자인 전략의 핵심이다. GV70는 제네시스(Genesis)의 다재다능한(Versatile) 차량이라는 의미가 담긴 이니셜 ‘GV’와 중형 차급을 뜻하는 ‘70’를 결합한 차명에서 알 수 있듯이 럭셔리 중형 SUV 시장을 겨냥하는 모델이다., Q. 전면부 디자인의 특징은 무엇인가? 전면부에서는 제네시스의 윙 엠블럼을 쿼드램프와 크레스트 그릴로 형상화한 윙 페이스(Wing Face) 디자인이 가장 큰 특징이다. 쿼드램프와 크레스트 그릴을 동일한 높이로 배열해 와이드한 느낌의 전면부 이미지를 구현했다. 또한 볼륨감과 입체감을 강조한 지-매트릭스(G-Matrix) 패턴을 라디에이터 그릴과 인테이크 그릴(범퍼 흡기구)에 적용해 제네시스만의 아이덴티티와 스포티한 감성을 전달한다. 아울러 앞범퍼 하단의 스키드 플레이트에서 시작되는 크롬 몰딩을 도어 하단까지 연장해 차량의 존재감을 키웠다. 이는 SUV의 성격과 스포티한 스타일을 동시에

In [34]:
genBlogDf['word_genesis'] = contentList

In [35]:
genBlogDf.head(10)

,magazine,type,category,title,link,date,contents,tag,word_genesis
0,HMGJ,B,Tech,"역동적인 아름다움으로 완성된 럭셔리 중형 SUV, 제네시스 GV70의 외장 디자인 ...",https://news.hmgjournal.com/Tech/%EC%A0%9C%EB%...,2020-12-04,", GV70는 제네시스 브랜드의 첫 번째 중형 SUV 모델이자 브랜드의 다섯 번째 ...",", HMG, HMG저널, HMGjournal, 제네시스, GV70, GV80, G8...","[, GV70는 제네시스 브랜드의 첫 번째 중형 SUV 모델이자 브랜드의 다섯 번째..."
1,HMGJ,B,Group,"제네시스의 두 번째 SUV, GV70의 디자인에 대한 해외 반응",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-11-03,", , 제네시스의 두 번째 SUV, GV70의 디자인에 대한 해외 반응, 제네시스의...",", HMG, HMG저널, HMGjournal, 제네시스, GV70, SUV, 모터트...","[, , 제네시스의 두 번째 SUV, GV70의 디자인에 대한 해외 반응, 제네시스..."
2,HMGJ,B,Group,"제네시스 '더 뉴 G70' 디자인 공개, 해외 언론과 소비자들의 뜨거운 반응",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-09-23,", , 제네시스 더 뉴 G70 디자인 공개, 해외 언론과 소비자들의 뜨거운 반응, ...",", HMG, HMG저널, HMGjournal, 제네시스, G70, 더 뉴 G70, ...","[, , 제네시스 더 뉴 G70 디자인 공개, 해외 언론과 소비자들의 뜨거운 반응,..."
3,HMGJ,B,Group,"오트 쿠튀르의 우아함을 담다, 제네시스 G90 스페셜 에디션 스타더스트",https://news.hmgjournal.com/Group-Story/%EC%98...,2020-07-03,", , 제네시스 G90 스타더스트, 제네시스 G90 스타더스트, 제네시스 G90 스...",", HMG, HMG저널, HMGjournal, 제네시스, G90, 스페셜 에디션, ...","[, , 제네시스 G90 스타더스트, 제네시스 G90 스타더스트, 제네시스 G90 ..."
4,HMGJ,B,Tech,"제네시스 G80, 디자이너가 말하는 역동적인 우아함",https://news.hmgjournal.com/Tech/%EC%A0%9C%EB%...,2020-03-30,", , 제네시스_G80_디자이너가_말하는_역동적인_우아함, 제네시스_G80_디자이너...",", HMG, HMG저널, HMGjournal, 제네시스, 제네시스 G80, G80,...","[, , 제네시스_G80_디자이너가_말하는_역동적인_우아함, 제네시스_G80_디자이..."
5,HMGJ,B,Group,"제네시스 신형 G80 디자인, 해외 호평 쏟아지는 이유는",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-03-10,", , 제네시스 The All-new G80, 전 세계 언론 매체와 소비자들의 주목...",", HMG, HMG저널, HMGjournal, 제네시스, G80, 디자인, 세단, ...","[, , 제네시스 The All-new G80, 전 세계 언론 매체와 소비자들의 주..."
6,HMGJ,B,Group,"제네시스, 우수한 품질로 프리미엄 브랜드의 자질을 입증하다",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-02-28,", , 제네시스, 우수한 품질로 프리미엄 브랜드의 자질을 입증하다, 제네시스, 우수...",", HMG, HMG저널, HMGjournal, 제네시스, J.D.파워, 품질내구조사...","[, , 제네시스, 우수한 품질로 프리미엄 브랜드의 자질을 입증하다, 제네시스, 우..."
7,HMGJ,B,Group,"나만을 위한 GV80, 제네시스의 맞춤형 주문 프로그램 '유어 제네시스'",https://news.hmgjournal.com/Group-Story/%EB%A7...,2020-02-12,", , 맞춤형_주문_프로그램_유어제네시스, 맞춤형_주문_프로그램_유어제네시스, 맞춤...",", 제네시스, GV80, 유어제네시스, 프리미엄브랜드, 컨피규레이터, 편의사양, 옵...","[, , 맞춤형_주문_프로그램_유어제네시스, 맞춤형_주문_프로그램_유어제네시스, 맞..."
8,HMGJ,B,Tech,컬러 디자이너가 말하는 제네시스 GV80의 매력,https://news.hmgjournal.com/Tech/%EC%BB%AC%EB%...,2020-01-16,", , 컬러 디자이너가 말하는 제네시스 GV80의 매력, 컬러 디자이너가 말하는 제...",", HMG, HMG저널, HMGjournal, 제네시스, GV80, SUV, 컬러,...","[, , 컬러 디자이너가 말하는 제네시스 GV80의 매력, 컬러 디자이너가 말하는 ..."
9,HMGJ,B,Tech,"우아한 차체에 빚어진 강인함, 디자이너가 말하는 제네시스 GV80",https://news.hmgjournal.com/Tech/%EC%9A%B0%EC%...,2020-01-15,", , GV80-EXT-Designer, GV80-EXT-Designer, GV80...",", GV80, 제네시스, 프리미엄 SUV, 대형 SUV, SUV, 자동차 디자인, ...","[, , GV80-EXT-Designer, GV80-EXT-Designer, GV8..."


In [33]:
emptyList = []

for i in range(len(genBlogDf)) :
    if len(genBlogDf.loc[i, 'word_genesis']) == 0 :
        emptyList.append(i)

In [35]:
genBlogDf = genBlogDf.drop(emptyList)
genBlogDf.head(10)

,magazine,type,category,title,link,date,contents,word_genesis
0,HMGJ,B,Tech,"역동적인 아름다움으로 완성된 럭셔리 중형 SUV, 제네시스 GV70의 외장 디자인 ...",https://news.hmgjournal.com/Tech/%EC%A0%9C%EB%...,2020-12-04,", GV70는 제네시스 브랜드의 첫 번째 중형 SUV 모델이자 브랜드의 다섯 번째 ...","[, GV70는 제네시스 브랜드의 첫 번째 중형 SUV 모델이자 브랜드의 다섯 번째..."
1,HMGJ,B,Group,"제네시스의 두 번째 SUV, GV70의 디자인에 대한 해외 반응",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-11-03,", , 제네시스의 두 번째 SUV, GV70의 디자인에 대한 해외 반응, 제네시스의...","[, , 제네시스의 두 번째 SUV, GV70의 디자인에 대한 해외 반응, 제네시스..."
2,HMGJ,B,Group,"제네시스 '더 뉴 G70' 디자인 공개, 해외 언론과 소비자들의 뜨거운 반응",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-09-23,", , 제네시스 더 뉴 G70 디자인 공개, 해외 언론과 소비자들의 뜨거운 반응, ...","[, , 제네시스 더 뉴 G70 디자인 공개, 해외 언론과 소비자들의 뜨거운 반응,..."
3,HMGJ,B,Group,"오트 쿠튀르의 우아함을 담다, 제네시스 G90 스페셜 에디션 스타더스트",https://news.hmgjournal.com/Group-Story/%EC%98...,2020-07-03,", , 제네시스 G90 스타더스트, 제네시스 G90 스타더스트, 제네시스 G90 스...","[, , 제네시스 G90 스타더스트, 제네시스 G90 스타더스트, 제네시스 G90 ..."
4,HMGJ,B,Tech,"제네시스 G80, 디자이너가 말하는 역동적인 우아함",https://news.hmgjournal.com/Tech/%EC%A0%9C%EB%...,2020-03-30,", , 제네시스_G80_디자이너가_말하는_역동적인_우아함, 제네시스_G80_디자이너...","[, , 제네시스_G80_디자이너가_말하는_역동적인_우아함, 제네시스_G80_디자이..."
5,HMGJ,B,Group,"제네시스 신형 G80 디자인, 해외 호평 쏟아지는 이유는",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-03-10,", , 제네시스 The All-new G80, 전 세계 언론 매체와 소비자들의 주목...","[, , 제네시스 The All-new G80, 전 세계 언론 매체와 소비자들의 주..."
6,HMGJ,B,Group,"제네시스, 우수한 품질로 프리미엄 브랜드의 자질을 입증하다",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-02-28,", , 제네시스, 우수한 품질로 프리미엄 브랜드의 자질을 입증하다, 제네시스, 우수...","[, , 제네시스, 우수한 품질로 프리미엄 브랜드의 자질을 입증하다, 제네시스, 우..."
7,HMGJ,B,Group,"나만을 위한 GV80, 제네시스의 맞춤형 주문 프로그램 '유어 제네시스'",https://news.hmgjournal.com/Group-Story/%EB%A7...,2020-02-12,", , 맞춤형_주문_프로그램_유어제네시스, 맞춤형_주문_프로그램_유어제네시스, 맞춤...","[, , 맞춤형_주문_프로그램_유어제네시스, 맞춤형_주문_프로그램_유어제네시스, 맞..."
8,HMGJ,B,Tech,컬러 디자이너가 말하는 제네시스 GV80의 매력,https://news.hmgjournal.com/Tech/%EC%BB%AC%EB%...,2020-01-16,", , 컬러 디자이너가 말하는 제네시스 GV80의 매력, 컬러 디자이너가 말하는 제...","[, , 컬러 디자이너가 말하는 제네시스 GV80의 매력, 컬러 디자이너가 말하는 ..."
9,HMGJ,B,Tech,"우아한 차체에 빚어진 강인함, 디자이너가 말하는 제네시스 GV80",https://news.hmgjournal.com/Tech/%EC%9A%B0%EC%...,2020-01-15,", , GV80-EXT-Designer, GV80-EXT-Designer, GV80...","[, , GV80-EXT-Designer, GV80-EXT-Designer, GV8..."


In [37]:
genBlogDf = genBlogDf.reset_index(drop = True)
genBlogDf.head(10)

,magazine,type,category,title,link,date,contents,word_genesis
0,HMGJ,B,Tech,"역동적인 아름다움으로 완성된 럭셔리 중형 SUV, 제네시스 GV70의 외장 디자인 ...",https://news.hmgjournal.com/Tech/%EC%A0%9C%EB%...,2020-12-04,", GV70는 제네시스 브랜드의 첫 번째 중형 SUV 모델이자 브랜드의 다섯 번째 ...","[, GV70는 제네시스 브랜드의 첫 번째 중형 SUV 모델이자 브랜드의 다섯 번째..."
1,HMGJ,B,Group,"제네시스의 두 번째 SUV, GV70의 디자인에 대한 해외 반응",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-11-03,", , 제네시스의 두 번째 SUV, GV70의 디자인에 대한 해외 반응, 제네시스의...","[, , 제네시스의 두 번째 SUV, GV70의 디자인에 대한 해외 반응, 제네시스..."
2,HMGJ,B,Group,"제네시스 '더 뉴 G70' 디자인 공개, 해외 언론과 소비자들의 뜨거운 반응",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-09-23,", , 제네시스 더 뉴 G70 디자인 공개, 해외 언론과 소비자들의 뜨거운 반응, ...","[, , 제네시스 더 뉴 G70 디자인 공개, 해외 언론과 소비자들의 뜨거운 반응,..."
3,HMGJ,B,Group,"오트 쿠튀르의 우아함을 담다, 제네시스 G90 스페셜 에디션 스타더스트",https://news.hmgjournal.com/Group-Story/%EC%98...,2020-07-03,", , 제네시스 G90 스타더스트, 제네시스 G90 스타더스트, 제네시스 G90 스...","[, , 제네시스 G90 스타더스트, 제네시스 G90 스타더스트, 제네시스 G90 ..."
4,HMGJ,B,Tech,"제네시스 G80, 디자이너가 말하는 역동적인 우아함",https://news.hmgjournal.com/Tech/%EC%A0%9C%EB%...,2020-03-30,", , 제네시스_G80_디자이너가_말하는_역동적인_우아함, 제네시스_G80_디자이너...","[, , 제네시스_G80_디자이너가_말하는_역동적인_우아함, 제네시스_G80_디자이..."
5,HMGJ,B,Group,"제네시스 신형 G80 디자인, 해외 호평 쏟아지는 이유는",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-03-10,", , 제네시스 The All-new G80, 전 세계 언론 매체와 소비자들의 주목...","[, , 제네시스 The All-new G80, 전 세계 언론 매체와 소비자들의 주..."
6,HMGJ,B,Group,"제네시스, 우수한 품질로 프리미엄 브랜드의 자질을 입증하다",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-02-28,", , 제네시스, 우수한 품질로 프리미엄 브랜드의 자질을 입증하다, 제네시스, 우수...","[, , 제네시스, 우수한 품질로 프리미엄 브랜드의 자질을 입증하다, 제네시스, 우..."
7,HMGJ,B,Group,"나만을 위한 GV80, 제네시스의 맞춤형 주문 프로그램 '유어 제네시스'",https://news.hmgjournal.com/Group-Story/%EB%A7...,2020-02-12,", , 맞춤형_주문_프로그램_유어제네시스, 맞춤형_주문_프로그램_유어제네시스, 맞춤...","[, , 맞춤형_주문_프로그램_유어제네시스, 맞춤형_주문_프로그램_유어제네시스, 맞..."
8,HMGJ,B,Tech,컬러 디자이너가 말하는 제네시스 GV80의 매력,https://news.hmgjournal.com/Tech/%EC%BB%AC%EB%...,2020-01-16,", , 컬러 디자이너가 말하는 제네시스 GV80의 매력, 컬러 디자이너가 말하는 제...","[, , 컬러 디자이너가 말하는 제네시스 GV80의 매력, 컬러 디자이너가 말하는 ..."
9,HMGJ,B,Tech,"우아한 차체에 빚어진 강인함, 디자이너가 말하는 제네시스 GV80",https://news.hmgjournal.com/Tech/%EC%9A%B0%EC%...,2020-01-15,", , GV80-EXT-Designer, GV80-EXT-Designer, GV80...","[, , GV80-EXT-Designer, GV80-EXT-Designer, GV8..."


In [38]:
len(genBlogDf)

136

In [39]:
len(emptyList)

196

In [40]:
len(genBlogDf) + len(emptyList)

332

In [36]:
genBlogDf.to_csv('../crawling/data/hmgj_genesis_crawling_fin.csv', sep = '|', index = False)

In [4]:
genBlogDf = pd.read_csv('./data/hmgj_genesis_crawling.csv', sep = '|')
genBlogDf.head()

,magazine,type,category,title,link,date,contents
0,HMGJ,B,Tech,"역동적인 아름다움으로 완성된 럭셔리 중형 SUV, 제네시스 GV70의 외장 디자인 ...",https://news.hmgjournal.com/Tech/%EC%A0%9C%EB%...,2020-12-04,", GV70는 제네시스 브랜드의 첫 번째 중형 SUV 모델이자 브랜드의 다섯 번째 ..."
1,HMGJ,B,Group,"제네시스의 두 번째 SUV, GV70의 디자인에 대한 해외 반응",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-11-03,", , 제네시스의 두 번째 SUV, GV70의 디자인에 대한 해외 반응, 제네시스의..."
2,HMGJ,B,Group,"제네시스 '더 뉴 G70' 디자인 공개, 해외 언론과 소비자들의 뜨거운 반응",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-09-23,", , 제네시스 더 뉴 G70 디자인 공개, 해외 언론과 소비자들의 뜨거운 반응, ..."
3,HMGJ,B,Group,"오트 쿠튀르의 우아함을 담다, 제네시스 G90 스페셜 에디션 스타더스트",https://news.hmgjournal.com/Group-Story/%EC%98...,2020-07-03,", , 제네시스 G90 스타더스트, 제네시스 G90 스타더스트, 제네시스 G90 스..."
4,HMGJ,B,Tech,"제네시스 G80, 디자이너가 말하는 역동적인 우아함",https://news.hmgjournal.com/Tech/%EC%A0%9C%EB%...,2020-03-30,", , 제네시스_G80_디자이너가_말하는_역동적인_우아함, 제네시스_G80_디자이너..."


In [19]:
tag = driver.find_elements_by_css_selector('#print > article > div.view_contents.printCancel > div > a')
tag[0].text

'현대자동차그룹'

In [21]:
tagList = []
for link in genBlogDf['link'] :
    driver.get(link)
    
    aTag = driver.find_elements_by_css_selector('#print > article > div.view_contents.printCancel > div > a')
    tag = ''
    for a in aTag :
        tag += ', ' + a.text
    
    tagList.append(tag)    
    
tagList

[', HMG, HMG저널, HMGjournal, 제네시스, GV70, GV80, G80, G70, 디자인, 디자이너, 익스테리어, 외장, Athletic, Elegance, 역동적인, 우아함, SUV',
 ', HMG, HMG저널, HMGjournal, 제네시스, GV70, SUV, 모터트렌드, 잘롭닉, 오토블로그',
 ', HMG, HMG저널, HMGjournal, 제네시스, G70, 더 뉴 G70, 모터트렌드, 오토위크, 모토 1, 로드앤트랙, 자동차 매체',
 ', HMG, HMG저널, HMGjournal, 제네시스, G90, 스페셜 에디션, STARDUST, 스타더스트, 베니티 페어, 아카데미 시상식 애프터 파티, 투 톤 디자인',
 ', HMG, HMG저널, HMGjournal, 제네시스, 제네시스 G80, G80, The All-new G80, 역동적인 우아함, G80 디자인, 럭셔리 대형 세단, 파라볼릭 라인, 애슬레틱 파워 라인, 쿼드램프, 크레스트 그릴, G-매트릭스',
 ', HMG, HMG저널, HMGjournal, 제네시스, G80, 디자인, 세단, 후륜_세단, 미국, 해외_반응, 모터트렌드, 카앤드라이버, 오토블로그, 모터1, 로드앤트랙, 잘롭닉',
 ', HMG, HMG저널, HMGjournal, 제네시스, J.D.파워, 품질내구조사, VDS, 신차품질조사, IQS, G70, G80, G90, Genesis',
 ', 제네시스, GV80, 유어제네시스, 프리미엄브랜드, 컨피규레이터, 편의사양, 옵션, BTO, BuildToOrder, YourGenesis',
 ', HMG, HMG저널, HMGjournal, 제네시스, GV80, SUV, 컬러, 소재, 컬러_디자인, 컬러_디자이너, 우드, 알루미늄, 지_매트릭스, 디자인, 디자이너, 프리미엄_SUV, 자동차_디자이너',
 ', GV80, 제네시스, 프리미엄 SUV, 대형 SUV, SUV, 자동차 디자인, 쿼드램프, 크레스트 그릴, 파라볼릭 라인, 지-메트릭스, Athletic Eleg

In [22]:
tagList

[', HMG, HMG저널, HMGjournal, 제네시스, GV70, GV80, G80, G70, 디자인, 디자이너, 익스테리어, 외장, Athletic, Elegance, 역동적인, 우아함, SUV',
 ', HMG, HMG저널, HMGjournal, 제네시스, GV70, SUV, 모터트렌드, 잘롭닉, 오토블로그',
 ', HMG, HMG저널, HMGjournal, 제네시스, G70, 더 뉴 G70, 모터트렌드, 오토위크, 모토 1, 로드앤트랙, 자동차 매체',
 ', HMG, HMG저널, HMGjournal, 제네시스, G90, 스페셜 에디션, STARDUST, 스타더스트, 베니티 페어, 아카데미 시상식 애프터 파티, 투 톤 디자인',
 ', HMG, HMG저널, HMGjournal, 제네시스, 제네시스 G80, G80, The All-new G80, 역동적인 우아함, G80 디자인, 럭셔리 대형 세단, 파라볼릭 라인, 애슬레틱 파워 라인, 쿼드램프, 크레스트 그릴, G-매트릭스',
 ', HMG, HMG저널, HMGjournal, 제네시스, G80, 디자인, 세단, 후륜_세단, 미국, 해외_반응, 모터트렌드, 카앤드라이버, 오토블로그, 모터1, 로드앤트랙, 잘롭닉',
 ', HMG, HMG저널, HMGjournal, 제네시스, J.D.파워, 품질내구조사, VDS, 신차품질조사, IQS, G70, G80, G90, Genesis',
 ', 제네시스, GV80, 유어제네시스, 프리미엄브랜드, 컨피규레이터, 편의사양, 옵션, BTO, BuildToOrder, YourGenesis',
 ', HMG, HMG저널, HMGjournal, 제네시스, GV80, SUV, 컬러, 소재, 컬러_디자인, 컬러_디자이너, 우드, 알루미늄, 지_매트릭스, 디자인, 디자이너, 프리미엄_SUV, 자동차_디자이너',
 ', GV80, 제네시스, 프리미엄 SUV, 대형 SUV, SUV, 자동차 디자인, 쿼드램프, 크레스트 그릴, 파라볼릭 라인, 지-메트릭스, Athletic Eleg

In [23]:
genBlogDf['tag'] = tagList

In [25]:
genBlogDf.head()

,magazine,type,category,title,link,date,contents,tag
0,HMGJ,B,Tech,"역동적인 아름다움으로 완성된 럭셔리 중형 SUV, 제네시스 GV70의 외장 디자인 ...",https://news.hmgjournal.com/Tech/%EC%A0%9C%EB%...,2020-12-04,", GV70는 제네시스 브랜드의 첫 번째 중형 SUV 모델이자 브랜드의 다섯 번째 ...",", HMG, HMG저널, HMGjournal, 제네시스, GV70, GV80, G8..."
1,HMGJ,B,Group,"제네시스의 두 번째 SUV, GV70의 디자인에 대한 해외 반응",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-11-03,", , 제네시스의 두 번째 SUV, GV70의 디자인에 대한 해외 반응, 제네시스의...",", HMG, HMG저널, HMGjournal, 제네시스, GV70, SUV, 모터트..."
2,HMGJ,B,Group,"제네시스 '더 뉴 G70' 디자인 공개, 해외 언론과 소비자들의 뜨거운 반응",https://news.hmgjournal.com/Group-Story/%EC%A0...,2020-09-23,", , 제네시스 더 뉴 G70 디자인 공개, 해외 언론과 소비자들의 뜨거운 반응, ...",", HMG, HMG저널, HMGjournal, 제네시스, G70, 더 뉴 G70, ..."
3,HMGJ,B,Group,"오트 쿠튀르의 우아함을 담다, 제네시스 G90 스페셜 에디션 스타더스트",https://news.hmgjournal.com/Group-Story/%EC%98...,2020-07-03,", , 제네시스 G90 스타더스트, 제네시스 G90 스타더스트, 제네시스 G90 스...",", HMG, HMG저널, HMGjournal, 제네시스, G90, 스페셜 에디션, ..."
4,HMGJ,B,Tech,"제네시스 G80, 디자이너가 말하는 역동적인 우아함",https://news.hmgjournal.com/Tech/%EC%A0%9C%EB%...,2020-03-30,", , 제네시스_G80_디자이너가_말하는_역동적인_우아함, 제네시스_G80_디자이너...",", HMG, HMG저널, HMGjournal, 제네시스, 제네시스 G80, G80,..."
